In [1]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.7/248.7 kB 3.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [50]:
import spotipy, mysecrets,requests,base64,webbrowser,json,time
from urllib.parse import urlencode
from spotipy.oauth2 import SpotifyClientCredentials 

In [55]:
client_id = mysecrets.client_id
client_secret = mysecrets.client_secret

auth_headers = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": "http://localhost:7777/callback",
    "scope": "user-library-read"
} 

In [56]:
webbrowser.open("https://accounts.spotify.com/authorize?" + urlencode(auth_headers))

True

In [62]:
code = "AQAGvCfW2uKDLHheeE3WLaUkL9MFlOYeq8gd4NrOmSzBWSrike_N7yZ7kIZIAflscrrFUCrnbqU17qqwHmT49WQqJ4YkNCCguyzCz-t1NWdy31csTOzDV7HkzCAE4aaydOSsR7SjrczVdVItuFgZRJBvZu_1vNXeINLEo1WYsyG75Sx_N4Ibpa0eNUBZw1JmMPuHApU"

In [63]:
encoded_credentials = base64.b64encode(client_id.encode() + b':' + client_secret.encode()).decode("utf-8")

token_headers = {
    "Authorization": "Basic " + encoded_credentials,
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "http://localhost:7777/callback"
}

r = requests.post("https://accounts.spotify.com/api/token", data = token_data, headers = token_headers)

In [66]:
token = r.json()["access_token"]

print(token)

BQCjJt0tHYqLpBLJk5RUwe4vV9rH3xRrrRH0eE2mGUbsIQE5BYhlu5cMbASD7zeOYHlHvla_GPmc4Pv7C7J2bK48cX-ic_PnGcss6AEZtTyaoZlg2OvqknOQ5bOanNklEKqv1A9a5ZQLjTVATfAo9f8UdiOxfm8zyQn2tJ1Kg-1qcLltG3PyK3RqpDBuB-jz


In [67]:
user_headers = { "Authorization": "Bearer " + token, "Content-Type": "application/json" }
user_params = {"limit": 5}

user_tracks_response = requests.get("https://api.spotify.com/v1/me/tracks", params=user_params, headers=user_headers)

print(user_tracks_response.json())

{'href': 'https://api.spotify.com/v1/me/tracks?offset=0&limit=5', 'items': [{'added_at': '2022-11-19T04:11:21Z', 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'}, 'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02', 'id': '06HL4z0CvFAxyc27GXpf02', 'name': 'Taylor Swift', 'type': 'artist', 'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}], 'available_markets': ['CA', 'US'], 'external_urls': {'spotify': 'https://open.spotify.com/album/34OkZVpuzBa9y40DCy0LPR'}, 'href': 'https://api.spotify.com/v1/albums/34OkZVpuzBa9y40DCy0LPR', 'id': '34OkZVpuzBa9y40DCy0LPR', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273332d85510aba3eb28312cfb2', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e02332d85510aba3eb28312cfb2', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d00004851332d85510aba3eb28312cfb2', 'width': 64}], 

In [69]:
playlist = requests.get("https://api.spotify.com/v1/me/playlists/playlist_id", params=user_params, headers=user_headers)
print(playlist.json())

{'error': {'status': 404, 'message': 'Service not found'}}


In [70]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [ ]:
playlist_id = sp.

In [ ]:
#extracting track ids

def get_track_id(playlist_id):
    music_id = []
    playlist = sp.playlist(playlist_id)
    for item in playlist['tracks']['items']:
        music_track = item['track']
        music_id_list.append(music_tracl['id'])
    return music_id_list

In [ ]:
#extracting track details

def get_track_data(track_id):
    meta = sp.track(track_id)
    track_details = {"name": meta['name'], "album": meta['album']['name'],
                     "artist": meta['album']['artists'][0]['name'],
                     "release_date":meta['album']['release_date'],
                     "duration_in_mins": round((meta['duration_ms'] * 0.001/60.0,2)}
    return track_details

In [ ]:
# looping over track ids

tracks = []
for i in range(len(track_ids)):
    time.sleep(.5)
    track = get_track_data(track_ids[i])
    tracks.append(track)

In [ ]:
#saving in JSON file
with open('spotify.json','w') as outfile:
    json.dump(tracks,outfile, indent=4)